# Build a Wind Turbine Dashboard

Welcome to our tutorial on building an interactive dashboard showcasing essential metrics from wind turbine manufacturers.

We'll demonstrate how to leverage:
- Sliders and dropdowns
- Interactive plots
- Data indicators
- Tables and layouts

## Requirements

First, install the required packages:

```bash
pip install hvplot pandas panel
```

or with conda:

```bash
conda install -y -c conda-forge hvplot pandas panel
```

```{note}
If you are using the Jupyter Book these packages are already installed.
```

## 1. Setup

Let's start by importing the necessary libraries and setting up our environment.

In [ ]:
import hvplot.pandas
import pandas as pd
import panel as pn

pn.extension("tabulator")

# Define styling
ACCENT = "teal"
styles = {
    "box-shadow": "rgba(50, 50, 93, 0.25) 0px 6px 12px -2px, rgba(0, 0, 0, 0.3) 0px 3px 7px -3px",
    "border-radius": "4px",
    "padding": "10px",
}

## 2. Data Extraction

Next, we'll load and prepare our wind turbine data.

In [ ]:
# this is a decorator that caches the result of the function
@pn.cache()  # only download data once
def get_data():
    return pd.read_csv("https://assets.holoviz.org/panel/tutorials/turbines.csv.gz")

source_data = get_data()

# Calculate key data ranges
min_year = int(source_data["p_year"].min())
max_year = int(source_data["p_year"].max())
top_manufacturers = (
    source_data.groupby("t_manu").p_cap.sum().sort_values().iloc[-10:].index.to_list()
)

## 3. Data Filtering

Create a function to filter our data based on user selections.

In [ ]:
def filter_data(t_manu, year):
    data = source_data[(source_data.t_manu == t_manu) & (source_data.p_year <= year)]
    return data

## 4. Interactive Controls

Create widgets for user interaction:

In [ ]:
# Create widgets
t_manu = pn.widgets.Select(
    name="Manufacturer",
    value="Vestas",
    options=sorted(top_manufacturers),
    description="The name of the manufacturer",
)
p_year = pn.widgets.IntSlider(name="Year", value=max_year, start=min_year, end=max_year)

# Create reactive dataframe
df = pn.rx(filter_data)(t_manu=t_manu, year=p_year)
count = df.rx.len()
total_capacity = df.t_cap.sum()
avg_capacity = df.t_cap.mean()
avg_rotor_diameter = df.t_rd.mean()

## 5. Visualization

Create the bar plot showing capacity changes over time.

In [ ]:
fig = (
    df[["p_year", "t_cap"]].groupby("p_year").sum() / 10**6
).hvplot.bar(
    title="Capacity Change",
    rot=90,
    ylabel="Capacity (GW)",
    xlabel="Year",
    xlim=(min_year, max_year),
    color=ACCENT,
)

## 6. Dashboard Components

Create the various components of our dashboard:

In [ ]:
# Create image pane
image = pn.pane.JPG("https://assets.holoviz.org/panel/tutorials/wind_turbines_sunset.png")

# Create indicators
indicators = pn.FlexBox(
    pn.indicators.Number(
        value=count, name="Count", format="{value:,.0f}", styles=styles
    ),
    pn.indicators.Number(
        value=total_capacity / 1e6,
        name="Total Capacity (GW)",
        format="{value:,.1f}",
        styles=styles,
    ),
    pn.indicators.Number(
        value=avg_capacity/1e3,
        name="Avg. Capacity (MW)",
        format="{value:,.1f}",
        styles=styles,
    ),
    pn.indicators.Number(
        value=avg_rotor_diameter,
        name="Avg. Rotor Diameter (m)",
        format="{value:,.1f}",
        styles=styles,
    ),
)

# Create plot and table components
plot = pn.pane.HoloViews(fig, sizing_mode="stretch_both", name="Plot")
table = pn.widgets.Tabulator(df, sizing_mode="stretch_both", name="Table")

## 7. Final Layout

Combine everything into our final dashboard layout:

In [ ]:
# Create tabs
tabs = pn.Tabs(
    plot, table, styles=styles, sizing_mode="stretch_width", height=500, margin=10
)

# Create dashboard
dashboard = pn.template.FastListTemplate(
    title="Wind Turbine Dashboard",
    sidebar=[image, t_manu, p_year],
    main=[pn.Column(indicators, tabs, sizing_mode="stretch_both")],
    main_layout=None,
    accent=ACCENT,
)

# dashboard.servable()

In [ ]:
from IPython.display import HTML

HTML('''
     <iframe src="https://panel-org-build-dashboard.hf.space" 
             frameborder="0" 
             style="width: 100%; height: 1000px; border: none;">
     </iframe>
     ''')

### Complete Code

```{include} ./scripts/build-dashboard.py
:code: python
```

To run the code, type the following command in your terminal:

```bash
panel serve build-dashboard.py --dev
```

## Key Features

Our dashboard includes:

1. **Interactive Filters**
   - Manufacturer selection
   - Year range slider

2. **Key Metrics**
   - Total turbine count
   - Total capacity
   - Average capacity
   - Average rotor diameter

3. **Visualizations**
   - Capacity change over time
   - Interactive data table

4. **Responsive Layout**
   - Sidebar for controls
   - Main area for content
   - Tabbed interface for plot/table